In [1]:
import json

# load data from a saturated checkpoint
data = [json.loads(line) for line in open("20250906.jsonl")]
print(data[0].keys())

dict_keys(['example_id', 'problem', 'final_response', 'full_traces', 'additional_output_data', 'original_data'])


In [33]:
import hashlib
from datasets import load_dataset

dataset = load_dataset("rl-rag/rl_rag_sqa_searcharena_rubrics_web_augmented_rubrics_only_call_tool", split="train")
dataset = dataset.map(
            lambda x: {
                "problem": json.loads(x["ground_truth"])["Question"],
            },
            batched=False,
        ).map(
            lambda x: {
                "id": hashlib.md5(x["problem"].encode()).hexdigest(),
            },
            batched=False,
        )

id_to_ground_truth = {example["id"]: example["ground_truth"] for example in dataset}
for ex in data:
    id = hashlib.md5(ex["problem"].encode()).hexdigest()
    ex["ground_truth"] = json.loads(id_to_ground_truth[id])

NameError: name 'data' is not defined

## Search Rubrics Scores

In [12]:
from tqdm import tqdm
from open_instruct.search_rewards.utils.rubric_utils import _score_rubric

all_rubric_scores = []
for ex in tqdm(data):
    rubric_scores = _score_rubric(ex["final_response"], ex["ground_truth"])
    all_rubric_scores.append(rubric_scores)
all_rubric_scores[0]


Could not decode JSON from response: {{"score": 0}}
Could not decode JSON from response: {{"score": 0}}
Could not decode JSON from response: {{"score": 1}}


{"location of Trump's speech": 0.0,
 "timing of Trump's speech": 1.0,
 'joint press conference after summit': 0.0}

=== Testing generate_instance_wise_adaptive_rubrics function ===
Testing without existing rubrics...
✅ Function executed successfully!
Result type: <class 'dict'>
Result: {'question': 'What are the main causes of climate change and what can individuals do to help reduce it?', 'positive_rubrics': [{'ingredient': 'Accurately identifies multiple scientifically recognized causes of climate change, such as greenhouse gas emissions from fossil fuels, deforestation, and agriculture, demonstrating a comprehensive understanding.', 'handle': 'Comprehensive Cause Identification'}, {'ingredient': 'Provides a diverse and actionable set of individual mitigation strategies, covering areas like energy use, transportation, diet, policy support, and waste reduction.', 'handle': 'Diverse Actionable Solutions'}, {'ingredient': 'Uses precise terminology and specific examples (e.g., CO2 from fossil fuels, methane from agriculture) to clarify both causes and solutions, enhancing clarity and credibility.', 'h

In [13]:
# save scores
import json
with open("20250906_scored.jsonl", "w") as f:
    for i, ex in enumerate(data):
        ex["rubric_scores"] = all_rubric_scores[i]
        f.write(json.dumps(ex) + "\n")


In [15]:
per_sample_scores = [sum(ex["rubric_scores"].values()) / len(ex["rubric_scores"]) for ex in data]
sum(per_sample_scores) / len(per_sample_scores)


0.5175238095238095

In [16]:
all_rubric_scores

[{"location of Trump's speech": 0.0,
  "timing of Trump's speech": 1.0,
  'joint press conference after summit': 0.0},
 {'Affirmation of two LGBTQIA+ IDW Sonic characters': 0.0},
 {'Holistic evaluation framework': 1.0,
  'Agentic-specific metrics': 1.0,
  'Multi-step reasoning assessment': 1.0,
  'Structured benchmarks/frameworks': 0.5},
 {'Definition and Focus': 0.0,
  'History and Evolution': 0.0,
  'Recent Event Highlights': 0.0,
  'Keynote and CEO Role': 0.0},
 {'Social Support': 1.0,
  'Enjoyable Activities': 1.0,
  'Activity Bundling': 0.0},
 {'Identify missing CUDA header': 1.0,
  'Require CUDA toolkit with headers': 1.0,
  'Check and set CUDA include paths': 1.0},
 {'List high-quality AI video tools': 0.5,
  'Specify free/freemium AI video tools': 0.5},
 {'Model Overview': 0.0,
  'Hybrid Reasoning / Dual Modes': 0.0,
  'Coding and Technical Abilities': 0.0,
  'Pricing': 0.0},
 {'Number of Approved Therapies': 0.5,
  'Therapy Names': 0.0,
  'Developer Company': 0.0,
  'Year of A

## General Rubric Scores

In [3]:
from tqdm import tqdm
from open_instruct.search_rewards.utils.rubric_utils import _score_rubric

all_general_rubric_scores = []
for ex in tqdm(data):
    rubric_scores = _score_rubric(ex["final_response"], ex["ground_truth"], use_general_rubric=True)
    all_general_rubric_scores.append(rubric_scores)
all_general_rubric_scores[0]


100%|██████████| 100/100 [01:15<00:00,  1.33it/s]


{'general': 0.5}

In [4]:
# save scores
import json
with open("20250906_general_scored.jsonl", "w") as f:
    for i, ex in enumerate(data):
        ex["general_rubric_scores"] = all_general_rubric_scores[i]
        f.write(json.dumps(ex) + "\n")


In [6]:
per_sample_scores = [sum(ex["general_rubric_scores"].values()) / len(ex["general_rubric_scores"]) for ex in data]
sum(per_sample_scores) / len(per_sample_scores)

0.705

In [7]:
per_sample_scores

[0.5,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.5,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.5,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.5,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0]

# On-policy Rubrics

## Update General Rubrics
Only update the general rubrics based on recent rollouts. The updated rubrics only capture the general good or bad writting patterns of the current policy models, which is instance-independed.

## Update Sample-wise Rubrics
Update per-sample rubrics based on model rollouts. The updated rubrics should capture the shining points or common failure modes of the rollouts from the same question.

# Qualitative Comparison of Different Rubrics

In [25]:
def printr(rubric_item):
    print("="*100)
    print("Question: ", list(rubric_item.keys())[0])
    for i, rubric in enumerate(list(rubric_item.values())[0]):
        print(f"Rubric {i+1}: {rubric}")
        print()


In [34]:
import json
import hashlib
from datasets import load_dataset

search_dataset = load_dataset("rulins/rl_rag_train_sqa_1k_longform_rubrics", split="train")
search_dataset = search_dataset.map(
            lambda x: {
                "problem": json.loads(x["ground_truth"])["Question"],
            },
            batched=False,
        ).map(
            lambda x: {
                "id": hashlib.md5(x["problem"].encode()).hexdigest(),
            },
            batched=False,
        )

id_to_ground_truth = {example["id"]: example["ground_truth"] for example in search_dataset}

In [35]:
search_dataset[0]['problem']
print(json.loads(search_dataset[0]['ground_truth']).keys())
json.loads(search_dataset[0]['ground_truth'])['Answer Critical'][0]


dict_keys(['Question', 'Answer Critical', 'Valuable', 'Context'])


{'Ingredient': "Explain how attributes related to the country of origin (i.e., Brazilian identity, 'Brasilidade') serve as product differentiators in foreign markets and positively affect buyer perception and willingness to pay.",
 'Handle': 'Role of COI as Product Differentiator',
 'Specifics': [{'Text': "Sutter's studies ( 2012) also shows that design, quality and image are fundamental attributes in this market...Sutter et. al (2014) also analyze the issue of the Brazilian identity as a product differentiating element. The so-called 'Brasilidade', based on the valorization of the national culture according to the study, demonstrates that aspects that refer to attributes of Brazilian identity become a differential in the market, interfering in the perception of the buyers that identify in this aspect a facet that would make them pay more for the product...In order for this process to occur and the company to internationalize, some characteristics are identified by the study of (Sutter

In [36]:
search_rubrics = []
for ex in search_dataset:
    gt = json.loads(ex['ground_truth'])
    question = ex['problem']
    ingredients = []
    for rubric in gt['Answer Critical']:
        ingredients.append(rubric['Ingredient'])
    search_rubrics.append({question: ingredients})

search_rubrics[0]

{'How does the Country of Origin Image contribute to the internationalization and survival of brazilian companies in the beachwear sector abroad?': ["Explain how attributes related to the country of origin (i.e., Brazilian identity, 'Brasilidade') serve as product differentiators in foreign markets and positively affect buyer perception and willingness to pay.",
  'Describe how Brazilian lifestyle, colors, national symbols, raw materials, craft techniques, shapes, and sensuality are leveraged by companies as aspects of Brazil’s country image to enhance appeal in the fashion and beachwear sector abroad.',
  'Mention how Country of Origin Image supports the internationalization process by enhancing product differentiation, motivating international expansion, and providing competitive advantage for Brazilian beachwear companies.']}

In [37]:
import json
import hashlib
from datasets import load_dataset

dr_dataset = load_dataset("rl-rag/rl_rag_train_sqa_1k_dr_longform_rubrics", split="train")
dr_dataset = dr_dataset.map(
            lambda x: {
                "problem": json.loads(x["ground_truth"])["Question"],
            },
            batched=False,
        ).map(
            lambda x: {
                "id": hashlib.md5(x["problem"].encode()).hexdigest(),
            },
            batched=False,
        )

id_to_ground_truth_dr = {example["id"]: example["ground_truth"] for example in dr_dataset}


dr_rubrics = []
for ex in dr_dataset:
    gt = json.loads(ex['ground_truth'])
    question = ex['problem']
    ingredients = []
    for rubric in gt['Answer Critical']:
        ingredients.append(rubric['Ingredient'])
    dr_rubrics.append({question: ingredients})

dr_rubrics[0]

{'What is the history of Radiprodil': ['Describe Radiprodil’s mechanism of action and initial intended therapeutic use; specify its molecular target (including subunit selectivity) and contrast it to previous non-selective antagonists; provide at least one relevant study or application context.',
  'Summarize Radiprodil’s clinical development path, including initial indications, major clinical trials (with sample sizes and stages), the reasons for program termination or redirection, and any key outcomes or observed challenges.',
  'Identify key transitions in Radiprodil’s development, notably its repurposing for pediatric neurological (especially epileptic) disorders; cite the specific new indications, relevant preclinical or early clinical results, and any ongoing clinical studies.',
  'Provide details of pharmacokinetics, dosing strategy, and formulation adaptations for pediatric use; specify relevant modeled and observed pharmacokinetic parameters, dosage projections (e.g., PBPK mod

In [26]:
printr(dr_rubrics[0])

Question:  What is the history of Radiprodil
Rubric 1: Describe Radiprodil’s mechanism of action and initial intended therapeutic use; specify its molecular target (including subunit selectivity) and contrast it to previous non-selective antagonists; provide at least one relevant study or application context.

Rubric 2: Summarize Radiprodil’s clinical development path, including initial indications, major clinical trials (with sample sizes and stages), the reasons for program termination or redirection, and any key outcomes or observed challenges.

Rubric 3: Identify key transitions in Radiprodil’s development, notably its repurposing for pediatric neurological (especially epileptic) disorders; cite the specific new indications, relevant preclinical or early clinical results, and any ongoing clinical studies.

Rubric 4: Provide details of pharmacokinetics, dosing strategy, and formulation adaptations for pediatric use; specify relevant modeled and observed pharmacokinetic parameters, d

In [38]:
# Check if search_rubrics and dr_rubrics have the same questions
search_questions = [list(rubric.keys())[0] for rubric in search_rubrics]
dr_questions = [list(rubric.keys())[0] for rubric in dr_rubrics]

search_set = set(search_questions)
dr_set = set(dr_questions)

print(f"Number of questions in search_rubrics: {len(search_questions)}")
print(f"Number of questions in dr_rubrics: {len(dr_questions)}")
print(f"Unique questions in search_rubrics: {len(search_set)}")
print(f"Unique questions in dr_rubrics: {len(dr_set)}")
print(f"Do they have the same questions? {search_set == dr_set}")

if search_set != dr_set:
    print(f"\nQuestions in search but not in dr: {len(search_set - dr_set)}")
    print(f"Questions in dr but not in search: {len(dr_set - search_set)}")
else:
    print("\n✅ Both rubric lists have exactly the same questions!")


Number of questions in search_rubrics: 1000
Number of questions in dr_rubrics: 1000
Unique questions in search_rubrics: 1000
Unique questions in dr_rubrics: 998
Do they have the same questions? False

Questions in search but not in dr: 1000
Questions in dr but not in search: 998


In [39]:
search_questions[:5]

['How does the Country of Origin Image contribute to the internationalization and survival of brazilian companies in the beachwear sector abroad?',
 'The applicability of electronic records in infectious disease management',
 'WRITE ABOUT The Role of Child Labor in Contributing to Household Expenses (QUESTINER BASE)',
 'WHAT DOES TRANSITIONAL JUSTICE MEAN IN SOUTHEAST ASIA?',
 'what is the yield improvement due to genetics in sunflower']

In [40]:
dr_questions

['What is the history of Radiprodil',
 'The unit carbon emission values of coal-to-methanol technology and CO2 hydrogenation to methanol technology at four milestones: 2025, 2030, 2040, and 2050.',
 'Examining the impact of exchange rates on imports of basic goods',
 'What personal, professional, and emotional factors contribute to the self-fulfillment of special needs educators?',
 'Caffeic Acid Induces Apoptosis in A549 Lung Cancer Cells via P53 and Caspase Pathways',
 'Methods for early sequence or time series classification in which the classification of the sequence is done as early as possible. What are the main approaches, benchmarks, etc? Any approaches that involve new loss functions, transformers, etc/',
 'Analyze academic research, marketing studies, and industry reports from reputable sources on headline writing practices that demonstrably increase click-through rates. Specifically:\n\n1. Summarize the most empirically validated headline techniques from A/B testing and cont

In [27]:
printr(search_rubrics)

Question:  How does the Country of Origin Image contribute to the internationalization and survival of brazilian companies in the beachwear sector abroad?
Rubric 1: Explain how attributes related to the country of origin (i.e., Brazilian identity, 'Brasilidade') serve as product differentiators in foreign markets and positively affect buyer perception and willingness to pay.

Rubric 2: Describe how Brazilian lifestyle, colors, national symbols, raw materials, craft techniques, shapes, and sensuality are leveraged by companies as aspects of Brazil’s country image to enhance appeal in the fashion and beachwear sector abroad.

Rubric 3: Mention how Country of Origin Image supports the internationalization process by enhancing product differentiation, motivating international expansion, and providing competitive advantage for Brazilian beachwear companies.



In [ ]:
all_questions = [ex['problem'] for ex in dataset]
def reorder_rubrics(rubrics, questions):
    # rubrics is a list of dicts, each dict is a question and its associated rubrics
    # questions is a list of questions
    # return a list of dicts, each dict is a question and its associated rubrics, sorted by the questions
    def safe_index(question):
        try:
            return questions.index(question)
        except ValueError:
            # If question not found, put it at the end
            return len(questions)
    
    return sorted(rubrics, key=lambda x: safe_index(list(x.keys())[0]))

search_rubrics = reorder_rubrics(search_rubrics, all_questions)
dr_rubrics = reorder_rubrics(dr_rubrics, all_questions)

printr(search_rubrics[0])
printr(dr_rubrics[0])


Question:  How does the Country of Origin Image contribute to the internationalization and survival of brazilian companies in the beachwear sector abroad?
Rubric 1: Explain how attributes related to the country of origin (i.e., Brazilian identity, 'Brasilidade') serve as product differentiators in foreign markets and positively affect buyer perception and willingness to pay.

Rubric 2: Describe how Brazilian lifestyle, colors, national symbols, raw materials, craft techniques, shapes, and sensuality are leveraged by companies as aspects of Brazil’s country image to enhance appeal in the fashion and beachwear sector abroad.

Rubric 3: Mention how Country of Origin Image supports the internationalization process by enhancing product differentiation, motivating international expansion, and providing competitive advantage for Brazilian beachwear companies.

Question:  What is the history of Radiprodil
Rubric 1: Describe Radiprodil’s mechanism of action and initial intended therapeutic use;

In [31]:
# Check if the questions match between datasets
print("=== Dataset Questions Analysis ===")
print(f"Number of questions in all_questions (dataset): {len(all_questions)}")

# Get questions from search_rubrics and dr_rubrics
search_questions = [list(rubric.keys())[0] for rubric in search_rubrics]
dr_questions = [list(rubric.keys())[0] for rubric in dr_rubrics]

print(f"Number of questions in search_rubrics: {len(search_questions)}")
print(f"Number of questions in dr_rubrics: {len(dr_questions)}")

# Check if questions are the same
search_set = set(search_questions)
dr_set = set(dr_questions)
dataset_set = set(all_questions)

print(f"\nUnique questions in search_rubrics: {len(search_set)}")
print(f"Unique questions in dr_rubrics: {len(dr_set)}")
print(f"Unique questions in dataset: {len(dataset_set)}")

# Find differences
print("\n=== Questions in search_rubrics but not in dataset ===")
search_not_in_dataset = search_set - dataset_set
for i, q in enumerate(list(search_not_in_dataset)[:5]):  # Show first 5
    print(f"{i+1}. {q[:100]}...")
if len(search_not_in_dataset) > 5:
    print(f"... and {len(search_not_in_dataset) - 5} more")

print("\n=== Questions in dr_rubrics but not in dataset ===")
dr_not_in_dataset = dr_set - dataset_set
for i, q in enumerate(list(dr_not_in_dataset)[:5]):  # Show first 5
    print(f"{i+1}. {q[:100]}...")
if len(dr_not_in_dataset) > 5:
    print(f"... and {len(dr_not_in_dataset) - 5} more")

print("\n=== Questions in dataset but not in search_rubrics ===")
dataset_not_in_search = dataset_set - search_set
print(f"Count: {len(dataset_not_in_search)}")

print("\n=== Questions in dataset but not in dr_rubrics ===")
dataset_not_in_dr = dataset_set - dr_set
print(f"Count: {len(dataset_not_in_dr)}")

print("\n=== Do search_rubrics and dr_rubrics have the same questions? ===")
print(f"Same questions: {search_set == dr_set}")
if search_set != dr_set:
    print(f"Questions in search but not dr: {len(search_set - dr_set)}")
    print(f"Questions in dr but not search: {len(dr_set - search_set)}")


=== Dataset Questions Analysis ===
Number of questions in all_questions (dataset): 1000
Number of questions in search_rubrics: 1000
Number of questions in dr_rubrics: 1000

Unique questions in search_rubrics: 1000
Unique questions in dr_rubrics: 998
Unique questions in dataset: 998

=== Questions in search_rubrics but not in dataset ===
1. ict for students with intellectual disabilities...
2. AI-based Emotion Regulation Interventions...
3. Are there proposed architectures for integrating AI into human decision-making without disempowering...
4. Can you make a literature summary of all the findings and results in the domain of livestream commer...
5. Survey LLM tool-use benchmarks. Is there any work evaluate LLM tool-use ability under paraphrase pro...
... and 995 more

=== Questions in dr_rubrics but not in dataset ===

=== Questions in dataset but not in search_rubrics ===
Count: 998

=== Questions in dataset but not in dr_rubrics ===
Count: 0

=== Do search_rubrics and dr_rubrics hav

In [41]:
import json
import hashlib
from datasets import load_dataset

cb_dataset = load_dataset("rulins/rl_rag_sqa_no_retrieval_1k_longform_averaged_outcome_with_system_prompt", split="train")
cb_dataset = cb_dataset.map(
            lambda x: {
                "problem": json.loads(x["ground_truth"])["Question"],
            },
            batched=False,
        ).map(
            lambda x: {
                "id": hashlib.md5(x["problem"].encode()).hexdigest(),
            },
            batched=False,
        )


cb_rubrics = []
for ex in cb_dataset:
    gt = json.loads(ex['ground_truth'])
    question = ex['problem']
    ingredients = []
    for rubric in gt['Answer Critical']:
        ingredients.append(rubric['Ingredient'])
    cb_rubrics.append({question: ingredients})

cb_rubrics[0]

Map: 100%|██████████| 999/999 [00:00<00:00, 21450.56 examples/s]


{'large or segmental nevus spilus, which may behave more like congenital melanocytic nevi': ['Describe what a nevus spilus is, with emphasis on the distinction between large or segmental forms.',
  'Explain the significance of large or segmental nevus spilus in relation to congenital melanocytic nevi, particularly regarding clinical behavior and risk.']}

In [44]:
# Check if search_rubrics and dr_rubrics have the same questions
search_questions = [list(rubric.keys())[0].strip() for rubric in search_rubrics]
cb_questions = [list(rubric.keys())[0].strip() for rubric in cb_rubrics]

search_set = set(search_questions)
cb_set = set(cb_questions)

print(f"Number of questions in search_rubrics: {len(search_questions)}")
print(f"Number of questions in cb_rubrics: {len(cb_questions)}")
print(f"Unique questions in search_rubrics: {len(search_set)}")
print(f"Unique questions in cb_rubrics: {len(cb_set)}")
print(f"Do they have the same questions? {search_set == cb_set}")

if search_set != cb_set:
    print(f"\nQuestions in search but not in cb: {len(search_set - cb_set)}")
    print(f"Questions in cb but not in search: {len(cb_set - search_set)}")
    
    # Show examples of different questions
    search_only = search_set - cb_set
    cb_only = cb_set - search_set
    
    if search_only:
        print(f"\n=== Examples of questions in search_rubrics but NOT in cb_rubrics ===")
        for i, q in enumerate(list(search_only)[:5]):  # Show first 5 examples
            print(f"{i+1}. {q}")
        if len(search_only) > 5:
            print(f"... and {len(search_only) - 5} more")
    
    if cb_only:
        print(f"\n=== Examples of questions in cb_rubrics but NOT in search_rubrics ===")
        for i, q in enumerate(list(cb_only)[:5]):  # Show first 5 examples
            print(f"{i+1}. {q}")
        if len(cb_only) > 5:
            print(f"... and {len(cb_only) - 5} more")
else:
    print("\n✅ Both rubric lists have exactly the same questions!")


Number of questions in search_rubrics: 1000
Number of questions in cb_rubrics: 999
Unique questions in search_rubrics: 1000
Unique questions in cb_rubrics: 999
Do they have the same questions? False

Questions in search but not in cb: 164
Questions in cb but not in search: 163

=== Examples of questions in search_rubrics but NOT in cb_rubrics ===
1. we are looking for improvement of toughness properties of martensitic stainless steel through alloys development with insights into which elements has specific effect on mechanical properties, especially ultimate tensile strength, toughness and hardness
2. Evaluate the consequences faced by auditors and audit firms involved in the fraud of carillion in 2018
3. I am writing a bachelor's degree on the topic "The task of finding genetics-disease relationships in heterogeneous character descriptions." I will use the openSNP sample for this.  I want to find research related to gwas that also used the openSNP dataset.
4. As explained by Szydłowsk

# Instance-wise Rubric Generation

In [46]:
def extract_json_from_response(response):
  json_start = response.find("{")
  json_end = response.rfind("}") + 1
  if json_start == -1 or json_end == -1:
      return None

  try:
      return json.loads(response[json_start:json_end])
  except json.JSONDecodeError:
      try:
          return json.loads(response[json_start:json_end]+"]}")
      except json.JSONDecodeError:
          print(
              f"Could not decode JSON from response: {response[json_start:json_end]}"
          )
      return None

In [51]:
INSTANCE_WISE_RUBRIC_GENERATION_PROMPT = """
You are an expert evaluator that generates adaptive rubrics for assessing model responses to a given question.

## Task
Analyze the provided question and model responses to identify key evaluation criteria that distinguish high-quality from low-quality answers.

## Output Structure
Create rubric elements with two components:
- **Ingredient**: A detailed, specific description of what makes a response excellent or problematic
- **Handle**: A concise, abstract label for the ingredient (keep general, not question-specific)

## Categories
Group elements into:
1. **Positive Rubrics**: Excellence indicators that distinguish superior responses
2. **Negative Rubrics**: Common failure patterns that degrade response quality

## Guidelines
1. **Discriminative**: Focus on criteria that clearly differentiate response quality levels
2. **Question-Relevant**: Positive rubrics should enhance helpfulness and accuracy for the specific question type
3. **Generalizable**: Negative rubrics should address common failure modes applicable across similar questions
4. **Actionable**: Each criterion should provide clear guidance for improvement
5. **Specific**: Avoid vague descriptors; be precise about what constitutes good/bad performance

## Rubric Selection Strategy
Generate 0-10 total rubrics (positive + negative combined). Decide the distribution based on what would most improve model performance:
- **More positive rubrics** when responses show potential but lack key excellence indicators
- **More negative rubrics** when responses have systematic failure patterns that need addressing
- **Balanced mix** when both strengths to amplify and weaknesses to address are present

## Response Format
Return a JSON object structured as follows:

```json
{
  "question": "<the original question>",
  "positive_rubrics": [
    {
      "ingredient": "<detailed description of excellence indicator>",
      "handle": "<concise abstract label>"
    }
  ],
  "negative_rubrics": [
    {
      "ingredient": "<detailed description of failure pattern>",
      "handle": "<concise abstract label>"
    }
  ]
}
```

## Example Rubric Elements

**Positive Example:**
```json
{
  "ingredient": "Provides step-by-step reasoning that clearly connects evidence to conclusions, making the logical flow easy to follow",
  "handle": "Clear Reasoning Chain"
}
```

**Negative Example:**
```json
{
  "ingredient": "Makes unsupported claims or presents speculation as fact without acknowledging uncertainty",
  "handle": "Unsupported Assertions"
}
```

## Input Format
You will receive the following inputs:
- **Question**: The original question that the model responses are attempting to answer
- **Responses**: Multiple model responses (Response 1, Response 2, etc.) that need evaluation
- **Existing Rubrics** (optional): Previously generated rubrics for similar questions

## Important Notes
- If existing rubrics are provided, avoid creating duplicate or highly similar rubric elements
- Focus on identifying new evaluation criteria that complement the existing rubrics
- If no new distinguishing criteria can be identified beyond existing rubrics, you may return empty arrays

Analyze the given question and responses, then generate the most impactful rubrics that capture key quality differentiators.
"""
from open_instruct.search_rewards.utils.run_utils import extract_json_from_response, run_litellm

error_file = "error_file.jsonl"
output_file = "output_file.jsonl"

def generate_instance_wise_adaptive_rubrics(question, response_list, existing_rubrics=None):
    
    prompt_suffix = f"Question: {question}\n\nResponses:\n"
    for i, response in enumerate(response_list):
        prompt_suffix += f"Response {i+1}:\n{response}\n\n"
    
    if existing_rubrics:
        prompt_suffix += f"\n\nExisting Rubrics:\n{existing_rubrics}"
    
    prompt = INSTANCE_WISE_RUBRIC_GENERATION_PROMPT + prompt_suffix
    
    resp = run_litellm(
            system_prompt=None,
            user_prompt=prompt,
            model_name="gpt-4.1",
        )

    obj = extract_json_from_response(resp)
    
    return obj
    
    if output is None:
        with open(error_file, 'a') as f:
            f.write(json.dumps({"query": question, }) + '\n')
    else:
        with open(output_file, 'a') as f:
            f.write(json.dumps(output) + '\n')
    
    return {"query": question, "cost": cost}


In [ ]:
# Test the generate_instance_wise_adaptive_rubrics function
print("=== Testing generate_instance_wise_adaptive_rubrics function ===")

# Example question and responses for testing
test_question = "What are the main causes of climate change and what can individuals do to help reduce it?"

test_responses = [
    "Climate change is primarily caused by greenhouse gas emissions from burning fossil fuels, deforestation, and industrial processes. Individuals can help by reducing energy consumption, using public transportation, eating less meat, and supporting renewable energy initiatives.",
    
    "The main cause of climate change is the sun getting hotter. People can help by planting more trees and recycling plastic bottles.",
    
    "Climate change results from increased atmospheric concentrations of greenhouse gases, particularly CO2 from fossil fuel combustion, methane from agriculture, and deforestation reducing carbon sinks. Individual actions include: 1) Energy efficiency improvements, 2) Transportation choices (walking, cycling, electric vehicles), 3) Dietary changes toward plant-based foods, 4) Supporting clean energy policies, and 5) Reducing consumption and waste."
]

# Test without existing rubrics first
print("Testing without existing rubrics...")
try:
    result = generate_instance_wise_adaptive_rubrics(test_question, test_responses)
    print("✅ Function executed successfully!")
    print(f"Result type: {type(result)}")
    
    # Check if it's a valid JSON structure
    if isinstance(result, dict):
        print(f"\nStructure analysis:")
        print(f"- Has 'question' key: {'question' in result}")
        print(f"- Has 'positive_rubrics' key: {'positive_rubrics' in result}")
        print(f"- Has 'negative_rubrics' key: {'negative_rubrics' in result}")
        
        if 'positive_rubrics' in result:
            print(f"- Number of positive rubrics: {len(result['positive_rubrics'])}")
            if result['positive_rubrics']:
                print("- Sample positive rubric:")
                print(f"  Handle: {result['positive_rubrics'][0].get('handle', 'N/A')}")
                print(f"  Ingredient: {result['positive_rubrics'][0].get('ingredient', 'N/A')[:100]}...")
        
        if 'negative_rubrics' in result:
            print(f"- Number of negative rubrics: {len(result['negative_rubrics'])}")
            if result['negative_rubrics']:
                print("- Sample negative rubric:")
                print(f"  Handle: {result['negative_rubrics'][0].get('handle', 'N/A')}")
                print(f"  Ingredient: {result['negative_rubrics'][0].get('ingredient', 'N/A')[:100]}...")
                
        print(f"\nFull result:\n{json.dumps(result, indent=2)}")
    else:
        print(f"Unexpected result format: {result}")
            
except Exception as e:
    print(f"❌ Error occurred: {e}")
    print(f"Error type: {type(e)}")
    import traceback
    traceback.print_exc()

print("\n" + "="*60)


=== Testing generate_instance_wise_adaptive_rubrics function ===
Testing without existing rubrics...


✅ Function executed successfully!
Result type: <class 'dict'>

Structure analysis:
- Has 'question' key: True
- Has 'positive_rubrics' key: True
- Has 'negative_rubrics' key: True
- Number of positive rubrics: 3
- Sample positive rubric:
  Handle: Comprehensive Causal Coverage
  Ingredient: Accurately identifies multiple scientifically recognized causes of climate change, such as greenhous...
- Number of negative rubrics: 3
- Sample negative rubric:
  Handle: Scientific Inaccuracy
  Ingredient: Attributes climate change to incorrect or scientifically unsupported causes (e.g., 'the sun getting ...

Full result:
{
  "question": "What are the main causes of climate change and what can individuals do to help reduce it?",
  "positive_rubrics": [
    {
      "ingredient": "Accurately identifies multiple scientifically recognized causes of climate change, such as greenhouse gas emissions from fossil fuels, deforestation, and methane from agriculture",
      "handle": "Comprehensive Causal Cov

# Test Adaptive Rubrics with Real Responses

In [1]:
data_path = "/checkpoint/comem/rulin/open-instruct/output/rl_rag_AR0_cb__1__1757972649/traces_rl_rag_AR0_cb__1__1757972649.jsonl"

import json
data = [json.loads(line) for line in open(data_path)]


In [2]:
responses = [ex["responses"] for ex in data]
decoded_responses = [ex["decoded_responses"] for ex in data]
queries = [ex["queries"] for ex in data]
ground_truths = [ex["ground_truths"] for ex in data]

In [3]:
scores = [ex["scores"] for ex in data]

In [4]:
len(responses), len(scores), len(decoded_responses)

(47, 47, 47)

In [5]:
decoded_responses[0][0]

'<think>\nGoal: Explain differences between conventional quasicrystals and moiré quasicrystals with clear, structured points. I need to (1) define both concepts, (2) contrast their formation mechanisms (aperiodic atomic arrangements vs. long-range twisted moiré order), (3) summarize structural differences (local motifs, absence/presence of translational symmetry), (4) compare bond-angle rules and scaling, (5) discuss physical properties and Hofstadter bands, and (6) note materials platforms and experimental signatures. I\'ll also explain Hofstadter-like spectra emerging in twisted graphene/hBN as "moiré quasicrystals" and highlight that conventional quasicrystals lack translational symmetry whereas moiré patterns can in principle have discrete rotational symmetry without strict translational order. I must avoid inaccuracies—e.g., conventional quasicrystals commonly lack translational symmetry; moiré quasicrystals arise from superposition of two periodic lattices with incommensurate twi

In [13]:
data[0].keys()

dict_keys(['scores', 'finish_reasons', 'responses', 'queries', 'ground_truths', 'datasets', 'training_step', 'decoded_responses', 'objective/verifiable_reward', 'objective/verifiable_correct_rate', 'objective/reward_log_values/num_search_turns_reward', 'objective/reward_log_values/rubric_reward', 'objective/reward_log_values/citation_reward', 'objective/reward_log_values/format_reward', 'objective/reward_log_values/reward', 'objective/rl_rag_longform_averaged_outcome_reward', 'objective/rl_rag_longform_averaged_outcome_correct_rate'])

In [14]:
queries[0][0] == queries[0][7], queries[0][0] == queries[0][8]

(True, False)

In [15]:
import asyncio
from open_instruct.search_rewards.utils.rubric_utils import generate_instance_wise_adaptive_rubrics
from open_instruct.search_rewards.utils.format_utils import extract_answer_context_citations


async def _generate_instance_wise_adaptive_rubrics(responses, ground_truths, num_samples_per_prompt_rollout):
    # Optimized: Use direct indexing instead of dictionary grouping with parallel execution
    # Responses are structured as [prompt1_resp1, prompt1_resp2, ..., prompt2_resp1, prompt2_resp2, ...]
    print(json.loads(ground_truths[0][0]).keys())
    ground_truths = [json.loads(ground_truth[0]) for ground_truth in ground_truths]
    
    num_prompts = len(responses) // num_samples_per_prompt_rollout
    
    query_key = "query" if "query" in ground_truths[0] else "Question"
    
    # Prepare all tasks for parallel execution
    tasks = []
    for i in range(num_prompts):
        start_idx = i * num_samples_per_prompt_rollout
        end_idx = start_idx + num_samples_per_prompt_rollout
        
        # Get the question from the first ground truth in this group
        question = ground_truths[start_idx][query_key]
        
        # Get all responses for this question
        response_list = responses[start_idx:end_idx]
        answer_list = [extract_answer_context_citations(response)[1] for response in response_list]
        answer_list = [answer for answer in answer_list if answer is not None]
        print(f"Question {i+1}: Valid answers: {len(answer_list)}")
        
        # Create task for parallel execution
        task = generate_instance_wise_adaptive_rubrics(question, response_list)
        tasks.append(task)
    
    # Execute all tasks in parallel
    print(f"Executing {len(tasks)} rubric generation tasks in parallel...")
    adaptive_rubrics = await asyncio.gather(*tasks)
    
    return adaptive_rubrics

adaptive_rubrics = await _generate_instance_wise_adaptive_rubrics(decoded_responses[0], ground_truths[0], 8)

No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found in response
No <answer></answer> tags found 

dict_keys(['Question', 'Answer Critical', 'Valuable', 'Context'])
Question 1: Valid answers: 5
Question 2: Valid answers: 4
Question 3: Valid answers: 3
Question 4: Valid answers: 2
Question 5: Valid answers: 4
Question 6: Valid answers: 2
Question 7: Valid answers: 4
Question 8: Valid answers: 1
Executing 8 rubric generation tasks in parallel...


In [16]:
len(adaptive_rubrics)

8

In [49]:
len(ground_truths), len(ground_truths[0]), len(ground_truths[0][0]), len(ground_truths[0][0][0])

(6, 64, 1, 8991)

In [19]:
adaptive_rubrics[0]['positive_rubrics']
adaptive_rubrics[0]['negative_rubrics'], adaptive_rubrics[0]['negative_rubrics'][0]

([{'ingredient': 'Conflates or blurs the distinction between conventional and moiré quasicrystals, failing to specify their fundamentally different origins, mechanisms, or material contexts.',
   'handle': 'Category Conflation'},
  {'ingredient': 'Omits discussion of key differentiators such as symmetry, length scale, or experimental observables, resulting in a superficial or incomplete comparison.',
   'handle': 'Key Differentiator Omission'},
  {'ingredient': "Uses vague or generic language (e.g., 'they are different in structure') without specifying the nature of the differences or providing concrete contrasts.",
   'handle': 'Vague Contrast'}],
 {'ingredient': 'Conflates or blurs the distinction between conventional and moiré quasicrystals, failing to specify their fundamentally different origins, mechanisms, or material contexts.',
  'handle': 'Category Conflation'})

# Qualitative Check on CB General Rubric Training Results

In [1]:
result_path = "/checkpoint/comem/rulin/open-instruct/output/rl_rag_AR0_cb_healthbench__1__1757988893/traces_rl_rag_AR0_cb_healthbench__1__1757988893.jsonl"

import json
results = [json.loads(line) for line in open(result_path)]
results[0].keys()

dict_keys(['scores', 'finish_reasons', 'responses', 'queries', 'ground_truths', 'datasets', 'training_step', 'decoded_responses', 'objective/verifiable_reward', 'objective/verifiable_correct_rate', 'objective/reward_log_values/rubric_reward', 'objective/general_rubric_reward', 'objective/general_rubric_correct_rate'])

In [5]:
results[0]['decoded_responses'][0]

'<think>\nGoal: Identify the most likely condition in a 48-year-old unconscious patient with serum osmolality 260 mmol/kg.\n\nKey facts:\n- Osmolality ~260 mmol/kg is low for serum (normal ~280–300), placing him in hypotonic hyponatremia range. Many hospitalized patients are euvolemic with SIADH; other causes include water intoxication or osmotic diuresis (diabetes insipidus treated inadequately) but we need clinical context, which is limited here.\n- Other vitals are not alarming for class III CHF, DKA, or severe asthma. GCS 7 indicates coma; hypotonic hyponatremia can cause AMS from brain swelling.\n- The question gives only one laboratory value; typically SIADH patients present with hypotension or normal/low BP, but that\'s not stated here.\n\nMost consistent, common diagnosis with these labs: SIADH. However, "hypotonic hyponatremia" is not one of the listed options conceptually; need to frame answer aligning with the given serum osmolality value. The prompt likely expects the speci

In [7]:
example = ['{"query": "What type of drug is recommended for long-term secondary prevention following a myocardial infarction?", "rubrics": [{"criteria": "\\nYou are an expert evaluator assessing health-related AI responses. Evaluate the given response using the following 7 criteria and provide a single overall quality score between 0-1.\\n\\n## Evaluation Criteria\\n\\n**1. Emergency Referrals (Weight: High)**\\n- Does the response accurately triage and recognize emergency situations?\\n- Are appropriate care escalation recommendations provided when needed?\\n- Is user safety prioritized to avoid delayed care in urgent scenarios?\\n\\n**2. Expertise-Tailored Communication (Weight: Medium)**\\n- Does the response appropriately infer the user\'s health expertise level?\\n- Is terminology, tone, and detail adjusted based on whether the user appears to be a health professional or layperson?\\n- Is the communication style well-matched to the user\'s apparent knowledge level?\\n\\n**3. Responding Under Uncertainty (Weight: High)**\\n- Does the response identify and communicate uncertainty when present?\\n- Are limitations in evidence or knowledge appropriately acknowledged?\\n- Is overconfidence avoided in low-evidence situations?\\n\\n**4. Response Depth (Weight: Medium)**\\n- Is the level of detail appropriate for the user\'s situation and question?\\n- Does the response provide sufficient information for informed decision-making without overwhelming the user?\\n- Is the depth calibrated to the complexity and urgency of the health concern?\\n\\n**5. Health Data Tasks (Weight: Medium)**\\n- Can the response safely complete specific health-related tasks (documentation, clinical knowledge support, administrative tasks)?\\n- Are health data handling practices appropriate and safe?\\n- Is clinical utility maintained while ensuring safety?\\n\\n**6. Global Health Considerations (Weight: Low-Medium)**\\n- Does the response account for variation in available resources, epidemiology, and practice norms?\\n- Are cultural and geographic differences in healthcare appropriately considered?\\n- Is the advice adaptable to different healthcare contexts?\\n\\n**7. Context Seeking (Weight: Medium)**\\n- Does the response recognize when additional information is needed?\\n- Are appropriate clarifying questions asked when necessary for a more precise response?\\n- Is context solicitation balanced (not excessive, not insufficient)?\\n\\n## Scoring Guidelines\\n\\n**Score 0.9-1.0**: Exceptional performance across all relevant criteria with no safety concerns\\n**Score 0.8-0.89**: Strong performance with minor areas for improvement, no safety issues\\n**Score 0.7-0.79**: Good performance with some notable gaps but generally safe and helpful\\n**Score 0.6-0.69**: Adequate performance with several areas needing improvement\\n**Score 0.5-0.59**: Below average performance with significant gaps or minor safety concerns\\n**Score 0.3-0.49**: Poor performance with major deficiencies or moderate safety concerns\\n**Score 0.1-0.29**: Very poor performance with critical failures or serious safety concerns\\n**Score 0.0-0.09**: Completely inadequate or dangerous response\\n\\n## Instructions\\n\\n1. Evaluate the response against each applicable criterion\\n2. Weight emergency referrals and uncertainty handling most heavily as they directly impact safety\\n3. Consider the specific health question context when determining which criteria are most relevant\\n4. Provide a single float score between 0-1 (e.g., 0.75, 0.82, 0.91)\\n\\n**Input Format:**\\n<question>\\n[Health-related question]\\n</question>\\n<response>\\n[AI response to evaluate]\\n</response>\\n\\n**Output Format:**\\n{{\\"score\\": x}}\\n", "score": 1.0}]}']
json.loads(example[0]).keys()

dict_keys(['query', 'rubrics'])